In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import itertools
import matplotlib.pyplot as plt

In [20]:
def preprocess_image_grayscale(img):
    """Convertit une image RGB en niveaux de gris et la duplique sur 3 canaux."""
    img = tf.image.rgb_to_grayscale(img)
    img = tf.image.grayscale_to_rgb(img)
    return img

In [21]:
# Chemin vers votre dossier de test
test_dir = '/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_struct_395img/val'

# Paramètres
batch_size = 32
target_size = (256, 256)

In [22]:
# Préparation des données de test
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image_grayscale)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)  # Important pour garder les étiquettes dans le bon ordre

Found 198 images belonging to 2 classes.


In [23]:
# Obtenir les vraies classes de l'ensemble de test
true_classes = test_generator.classes

In [24]:
import os
# Après avoir créé `test_generator` avec `flow_from_directory`
image_paths = test_generator.filenames

# Liste pour stocker tous les chemins d'images trouvés
list_img_path = []

# Afficher les chemins complets
full_paths = [os.path.join(test_dir, path) for path in image_paths]
for path in full_paths:
    # print(path)
    list_img_path.append(path)

In [25]:
len(list_img_path)

198

In [26]:
source = '/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Models/Main_classification_model'
list_path = []
for i in os.listdir(source):
    model_path = os.path.join(source,i)
    list_path.append(model_path)

In [27]:
h5_models = [path for path in list_path if path.endswith('.h5')]

In [28]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score

Comparaison model TF

In [29]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.backend import clear_session


# Assurez-vous que `h5_models` est bien défini et contient les chemins vers vos modèles .h5
# Exemple:
# h5_models = ['/chemin/vers/modele1.h5', '/chemin/vers/modele2.h5', ...]

# Votre générateur de test est déjà défini (test_generator)

# Liste pour stocker l'accuracy de chaque modèle
acc_list = []

for model_path in h5_models:
    # Chargement du modèle
    model = tf.keras.models.load_model(model_path)
    
    # Prédiction sur l'ensemble de test
    predictions = model.predict(test_generator, steps=np.ceil(test_generator.samples / test_generator.batch_size), verbose=1)
    
    # Convertir les probabilités en classes prédites basées sur un seuil de 0.5 pour une classification binaire
    predicted_classes = np.where(predictions.squeeze() > 0.5, 1, 0)
    
    # Obtenir les vraies classes de l'ensemble de test
    true_classes = test_generator.classes
    
    # Calculer l'accuracy
    accuracy = accuracy_score(true_classes, predicted_classes)
    
    # Afficher l'accuracy
    print(f'Model {model_path}: Accuracy = {accuracy}')
    
    # Ajouter l'accuracy à la liste
    acc_list.append(accuracy)
    clear_session()

# Afficher les accuracies de tous les modèles testés
print("Accuracies of all models:", acc_list)

: 

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

Comparaison model Pytorch

In [ ]:
import os
import torch
from ultralytics import YOLO
from sklearn.metrics import accuracy_score

In [ ]:
yolo_paths = [
    '/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Models/Main_classification_model/yolo_class_training_865img'
]

In [ ]:
# Liste pour stocker les chemins des fichiers .pt
pt_models = []

# Collecter les chemins des modèles .pt
for path in yolo_paths:
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.pt'):
                pt_models.append(os.path.join(root, file))

In [ ]:
# Évaluer chaque modèle YOLO
for model_path in pt_models:
    # Charger le modèle YOLO
    model_yolo = YOLO(model_path)
    
    # Prédire avec YOLO sur l'ensemble de test
    results_yolo = model_yolo(list_img_path)
    predicted_classes_yolo = [result.probs.top1 for result in results_yolo]

    # Calculer l'accuracy de YOLO
    accuracy_yolo = accuracy_score(true_classes, predicted_classes_yolo)
    print(f'YOLO Model {model_path}: Accuracy = {accuracy_yolo}')
    
    # Ajouter l'accuracy de YOLO à la liste des accuracies
    acc_list.append(accuracy_yolo)


0: 224x224 Martensite_microstructure_img_cropped_V3_manualy_cleaned_yolo 0.99, pearlite_microstructure_img_cropped_V3_cleaned_yolo 0.01, 1: 224x224 Martensite_microstructure_img_cropped_V3_manualy_cleaned_yolo 0.99, pearlite_microstructure_img_cropped_V3_cleaned_yolo 0.01, 2: 224x224 Martensite_microstructure_img_cropped_V3_manualy_cleaned_yolo 0.93, pearlite_microstructure_img_cropped_V3_cleaned_yolo 0.07, 3: 224x224 Martensite_microstructure_img_cropped_V3_manualy_cleaned_yolo 0.96, pearlite_microstructure_img_cropped_V3_cleaned_yolo 0.04, 4: 224x224 Martensite_microstructure_img_cropped_V3_manualy_cleaned_yolo 0.99, pearlite_microstructure_img_cropped_V3_cleaned_yolo 0.01, 5: 224x224 Martensite_microstructure_img_cropped_V3_manualy_cleaned_yolo 0.90, pearlite_microstructure_img_cropped_V3_cleaned_yolo 0.10, 6: 224x224 Martensite_microstructure_img_cropped_V3_manualy_cleaned_yolo 0.77, pearlite_microstructure_img_cropped_V3_cleaned_yolo 0.23, 7: 224x224 Martensite_microstructure_img

FUll comparaison 

In [ ]:
# Extraire des noms simplifiés à partir des chemins complets des modèles pour l'affichage
# Pour chaque modèle TensorFlow, extrayez un nom simplifié et ajoutez-le à model_names
model_names = [path.split('/')[-1] for path in h5_models]

# Faites de même pour les modèles YOLO, en extrayant un nom simplifié pour chaque modèle .pt
model_names += [path.split('/')[-1] for path in pt_models]

# Assurez-vous que model_names et acc_list sont correctement peuplés comme décrit ci-dessus
assert len(acc_list) == len(model_names), "La longueur de acc_list doit correspondre au nombre de modèles dans model_names."

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration de Seaborn pour le style du graphique
sns.set(style="whitegrid")

# Chemin vers votre dossier de test
title_segments = '/'.join(test_dir.split('/')[-2:])

# Préparation des données pour Seaborn
data = {'Model Names': model_names, 'Accuracy': acc_list}
data_plot = pd.DataFrame(data)

# Trier les données par accuracy pour un meilleur visuel
data_plot = data_plot.sort_values('Accuracy', ascending=True)

# Modifier les noms des modèles pour ajouter un saut de ligne
# Vous pouvez ajuster cette logique en fonction de la structure spécifique de vos noms de modèles
data_plot['Model Names'] = data_plot['Model Names'].apply(lambda x: '\n'.join([x[:len(x)//2], x[len(x)//2:]]))

# Création du graphique
plt.figure(figsize=(15, 10))
palette = sns.color_palette("viridis", len(data_plot['Model Names']))
barplot = sns.barplot(x='Accuracy', y='Model Names', data=data_plot, palette=palette)

plt.xlim(0, 1)
plt.title(f'Model Accuracy Comparison - {title_segments}', fontsize=16, fontweight='bold')
plt.xlabel('Accuracy', fontsize=14, fontweight='bold')
plt.ylabel('')

# Taille des ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=10)  # Ajustez si nécessaire

# Ajouter l'accuracy numérique sur chaque barre de manière visible
for index, value in enumerate(data_plot['Accuracy']):
    plt.text(value, index, f' {value:.2f}', color='black', va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()


NameError: name 'test_dir' is not defined